In [1]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

### Imports and setup

In [2]:
%run RandomAlgorithm.ipynb import *
%run Util.ipynb import *

In [3]:
import chess
import math
import random

### Set up evaluation function parameters

In [4]:
piece_values = {
    'p': 100,   # Pawns
    'n': 320,   # Knights
    'b': 330,   # Bishops
    'r': 500,   # Rooks
    'q': 900,   # Queen
    'k': 20000  # King
}

piece_square_tables = {
    'p': [ # Pawns
        [   0,   0,   0,   0,   0,   0,   0,   0 ],
        [  50,  50,  50,  50,  50,  50,  50,  50 ],
        [  10,  10,  20,  30,  30,  20,  10,  10 ],
        [   5,   5,  10,  25,  25,  10,   5,   5 ],
        [   0,   0,   0,  20,  20,   0,   0,   0 ],
        [   5,  -5, -10,   0,   0, -10,  -5,   5 ],
        [   5,  10,  10, -20, -20,  10,  10,   5 ],
        [   0,   0,   0,   0,   0,   0,   0,   0 ]
    ],

    'n': [ # Knights
        [ -50, -40, -30, -30, -30, -30, -40, -50 ],
        [ -40, -20,   0,   0,   0,   0, -20, -40 ],
        [ -30,   0,  10,  15,  15,  10,   0, -30 ],
        [ -30,   5,  15,  20,  20,  15,   5, -30 ],
        [ -30,   0,  15,  20,  20,  15,   0, -30 ],
        [ -30,   5,  10,  15,  15,  10,   5, -30 ],
        [ -40, -20,   0,   5,   5,   0, -20, -40 ],
        [ -50, -40, -30, -30, -30, -30, -40, -50 ]
    ],

    'b': [ # Bishops
        [ -20, -10, -10, -10, -10, -10, -10, -20 ],
        [ -10,   0,   0,   0,   0,   0,   0, -10 ],
        [ -10,   0,   5,  10,  10,   5,   0, -10 ],
        [ -10,   5,   5,  10,  10,   5,   5, -10 ],
        [ -10,   0,  10,  10,  10,  10,   0, -10 ],
        [ -10,  10,  10,  10,  10,  10,  10, -10 ],
        [ -10,   5,   0,   0,   0,   0,   5, -10 ],
        [ -20, -10, -10, -10, -10, -10, -10, -20 ]
    ],

    'r': [ # Rooks
        [   0,   0,   0,   0,   0,   0,   0,   0 ],
        [   5,  10,  10,  10,  10,  10,  10,   5 ],
        [  -5,   0,   0,   0,   0,   0,   0,  -5 ],
        [  -5,   0,   0,   0,   0,   0,   0,  -5 ],
        [  -5,   0,   0,   0,   0,   0,   0,  -5 ],
        [  -5,   0,   0,   0,   0,   0,   0,  -5 ],
        [  -5,   0,   0,   0,   0,   0,   0,  -5 ],
        [   0,   0,   0,   5,   5,   0,   0,   0 ]
    ],

    'q': [ # Queen
        [ -20, -10, -10,  -5,  -5, -10, -10, -20 ],
        [ -10,   0,   0,   0,   0,   0,   0, -10 ],
        [ -10,   0,   5,   5,   5,   5,   0, -10 ],
        [  -5,   0,   5,   5,   5,   5,   0,  -5 ],
        [   0,   0,   5,   5,   5,   5,   0,  -5 ],
        [ -10,   5,   5,   5,   5,   5,   0, -10 ],
        [ -10,   0,   5,   0,   0,   0,   0, -10 ],
        [ -20, -10, -10,  -5,  -5, -10, -10, -20 ]
    ],

    'km': [ # King (middle game)
        [ -30, -40, -40, -50, -50, -40, -40, -30 ],
        [ -30, -40, -40, -50, -50, -40, -40, -30 ],
        [ -30, -40, -40, -50, -50, -40, -40, -30 ],
        [ -30, -40, -40, -50, -50, -40, -40, -30 ],
        [ -20, -30, -30, -40, -40, -30, -30, -20 ],
        [ -10, -20, -20, -20, -20, -20, -20, -10 ],
        [  20,  20,   0,   0,   0,   0,  20,  20 ],
        [  20,  30,  10,   0,   0,  10,  30,  20 ]
    ],

    'ke': [ # King (end game)
        [ -50, -40, -30, -20, -20, -30, -40, -50 ],
        [ -30, -20, -10,   0,   0, -10, -20, -30 ],
        [ -30, -10,  20,  30,  30,  20, -10, -30 ],
        [ -30, -10,  30,  40,  40,  30, -10, -30 ],
        [ -30, -10,  30,  40,  40,  30, -10, -30 ],
        [ -30, -10,  20,  30,  30,  20, -10, -30 ],
        [ -30, -30,   0,   0,   0,   0, -30, -30 ],
        [ -50, -30, -30, -30, -30, -30, -30, -50 ]
    ]
}

In [5]:
def get_best_move(self) -> chess.Move:
    # return max(board.legal_moves, key=lambda move: get_move_score(board, move))

    # Using alpha beta search algorithm
    _, move = self.get_best_move_alphabeta(alpha=-100000000, beta=100000000, ai_turn=True, iteration=0, max_iterations=5)
    return move

chess.Board.get_best_move = get_best_move

In [6]:
def get_best_move_minimax(self, ai_turn: bool, iteration: int, max_iterations: int) -> (int, chess.Move):
    # If the game has finished, return a positive score if we won, or a negative score if we lost.
    # This score is higher than any possible score in the evaluation function, because wining or losing
    # the game is more impactful than any other board state.
    # If the game ended in a draw, return a neutral score (0).
    outcome = self.outcome()
    if outcome is not None:
        if outcome.winner == chess.WHITE:
            return 1000000, None
        if outcome.winner == chess.BLACK:
            return -1000000, None
        return 0, None # Draw

    # If the search has hit its max depth (max_iterations), return the current evaluation score for the AI
    if iteration >= max_iterations:
        return self.get_ai_score(), None

    # If the game has not finished, check additional moves using the minimax algorithm
    best_score, best_move = 10000000 * (-1 if ai_turn else 1), None
    for move in self.legal_moves:
        self.push(move)
        score_after_move, _ = self.get_best_move_minimax(not ai_turn, iteration + 1, max_iterations)
        if (ai_turn and score_after_move > best_score) or (not ai_turn and score_after_move < best_score):
            best_score = score_after_move
            best_move = move
        self.pop()

    return best_score, best_move

chess.Board.get_best_move_minimax = get_best_move_minimax

In [7]:
def get_best_move_alphabeta(self, alpha: int, beta: int, ai_turn: bool, iteration: int, max_iterations: int) -> (int, chess.Move):
    # If the game has finished, return a positive score if we won, or a negative score if we lost.
    # This score is higher than any possible score in the evaluation function, because wining or losing
    # the game is more impactful than any other board state.
    # If the game ended in a draw, return a neutral score (0).
    outcome = self.outcome()
    if outcome is not None:
        if outcome.winner == chess.WHITE:
            return 1000000, None
        if outcome.winner == chess.BLACK:
            return -1000000, None
        return 0, None # Draw

    # If the search has hit its max depth (max_iterations), return the current evaluation score for the AI
    if iteration >= max_iterations:
        return self.get_ai_score(), None

    # If the game has not finished, check additional moves using the minimax algorithm
    best_score, best_move = 10000000 * (-1 if ai_turn else 1), None
    for move in self.legal_moves:
        self.push(move)
        score_after_move, _ = self.get_best_move_alphabeta(alpha, beta, not ai_turn, iteration + 1, max_iterations)
        if (ai_turn and score_after_move > best_score) or (not ai_turn and score_after_move < best_score):
            best_score = score_after_move
            best_move = move
        self.pop()

        if ai_turn:
            if best_score >= beta:
                return best_score, move
            if best_score > alpha:
                alpha = best_score
        else:
            if best_score <= alpha:
                return best_score, move
            if best_score < beta:
                beta = best_score

    return best_score, best_move

chess.Board.get_best_move_alphabeta = get_best_move_alphabeta

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d6ce9acd-52c5-4422-904d-8424da19408b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>